In [681]:
import numpy as np

In [682]:
def generate(filename):
    file = open(filename)
    lines = file.readlines()
    
    X = []
    y = []
    
    
    
    for line in lines:
        var = line.split()
        X.append([float(x) for x in var[:-1]])
        y.append(int(var[-1]))
        
    unique = len(np.unique(y))
    
    Y = []
    for target in y:
        instance = np.zeros(unique)
        instance[target-1] = 1
        Y.append(instance)
        

    X = np.array(X)
    Y = np.array(Y)
    
    file.close()
    
    return X,np.array(Y)

In [683]:
X_train,Y_train = generate('trainNN.txt')
X_test, Y_test  =  generate('testNN.txt')

In [684]:
np.random.seed(42)
class L_Layer_NN:
    def __init__(self,L,k):
        self.L = L
        self.k = k
        self.W = []
        self.W.append([])
        self.k = np.insert(self.k,0,values=0,axis=0)
        print(self.k)
    
    def sigmoid(self,z):
        return 1/(1 + np.exp(-z))

    def derivative(self,z):
        return self.sigmoid(z)*(1-self.sigmoid(z))
    
    def fit(self,X_train,Y_train,max_epoch=1000):
        X_train = np.insert(X_train,0,values=1,axis=1)
        N = len(X_train)
        
        for i in range(1,self.L+1):
            if i == 1:
                self.W.append(np.random.uniform(-1,+1,(self.k[i],X_train.shape[1])))
            else:
                self.W.append(np.random.uniform(-1,+1,(self.k[i],self.k[i-1]+1)))
        
        for epoch in range(max_epoch):
            
            correct = 0
            J = 0
            
            for i in range(N):
                instance = X_train[i]
                
                layer_wise_output = []
                layer_wise_output.append(instance)
                
                v = []
                v.append([])
                
                ## forward propagation
                
                for r in range(1,self.L+1):
                    instance = instance.reshape(-1,1)
                    v_r = np.dot(self.W[r],instance).ravel()
                    y_r = self.sigmoid(v_r)
                    y_r = np.insert(y_r,0,values=1,axis=0)
                    v.append(v_r)
                    layer_wise_output.append(y_r)
                    instance = y_r
                    
                J += np.sum((layer_wise_output[self.L][1:]-Y_train[i])**2,axis=0)
                
                if np.argmax(layer_wise_output[self.L][1:]) == np.argmax(Y_train[i]):
                    correct += 1
                
                ## backpropagation
                
                delta_L = (layer_wise_output[self.L][1:]-Y_train[i])*self.derivative(v[self.L])
                print(self.W)
                
                ## correct till here
                
                delta = []
                delta.append([])
                
                for r in range(1,self.L+1):
                    if r == self.L:
                        delta.append(delta_L)
                    else:
                        delta.append(np.zeros(self.k[r]))
                
                
                for r in reversed(range(1,self.L)):
                    delta_r = []
                    for j in range(self.k[r]):
                        total = 0 
                        for p in range(self.k[r+1]):
                            total += delta[r+1][p]*self.W[r+1][p][j+1]
                        delta_r.append(total*self.derivative(v[r][j]))
                    delta[r] = np.array(delta_r)
                '''
                for r in range(1,self.L+1):
                    for j in range(self.k[r]):
                        self.W[r][j] = self.W[r][j] - 0.03*(delta[r][j]*layer_wise_output[r-1])
                '''
            break
            
            if epoch % 100 == 0:
                print(correct/N)
                print(J)
                print()
        pass

In [685]:
output_neurons = Y_train.shape[1] 

model = L_Layer_NN(2,[4,output_neurons])
model.fit(X_train,Y_train)

[0 4 4]
[[], array([[-0.25091976,  0.90142861,  0.46398788,  0.19731697, -0.68796272],
       [-0.68801096, -0.88383278,  0.73235229,  0.20223002,  0.41614516],
       [-0.95883101,  0.9398197 ,  0.66488528, -0.57532178, -0.63635007],
       [-0.63319098, -0.39151551,  0.04951286, -0.13610996, -0.41754172]]), array([[ 0.22370579, -0.72101228, -0.4157107 , -0.26727631, -0.08786003],
       [ 0.57035192, -0.60065244,  0.02846888,  0.18482914, -0.90709917],
       [ 0.2150897 , -0.65895175, -0.86989681,  0.89777107,  0.93126407],
       [ 0.6167947 , -0.39077246, -0.80465577,  0.36846605, -0.11969501]])]
[[], array([[-0.25091976,  0.90142861,  0.46398788,  0.19731697, -0.68796272],
       [-0.68801096, -0.88383278,  0.73235229,  0.20223002,  0.41614516],
       [-0.95883101,  0.9398197 ,  0.66488528, -0.57532178, -0.63635007],
       [-0.63319098, -0.39151551,  0.04951286, -0.13610996, -0.41754172]]), array([[ 0.22370579, -0.72101228, -0.4157107 , -0.26727631, -0.08786003],
       [ 0.570

[[], array([[-0.25091976,  0.90142861,  0.46398788,  0.19731697, -0.68796272],
       [-0.68801096, -0.88383278,  0.73235229,  0.20223002,  0.41614516],
       [-0.95883101,  0.9398197 ,  0.66488528, -0.57532178, -0.63635007],
       [-0.63319098, -0.39151551,  0.04951286, -0.13610996, -0.41754172]]), array([[ 0.22370579, -0.72101228, -0.4157107 , -0.26727631, -0.08786003],
       [ 0.57035192, -0.60065244,  0.02846888,  0.18482914, -0.90709917],
       [ 0.2150897 , -0.65895175, -0.86989681,  0.89777107,  0.93126407],
       [ 0.6167947 , -0.39077246, -0.80465577,  0.36846605, -0.11969501]])]
[[], array([[-0.25091976,  0.90142861,  0.46398788,  0.19731697, -0.68796272],
       [-0.68801096, -0.88383278,  0.73235229,  0.20223002,  0.41614516],
       [-0.95883101,  0.9398197 ,  0.66488528, -0.57532178, -0.63635007],
       [-0.63319098, -0.39151551,  0.04951286, -0.13610996, -0.41754172]]), array([[ 0.22370579, -0.72101228, -0.4157107 , -0.26727631, -0.08786003],
       [ 0.57035192, -

       [ 0.6167947 , -0.39077246, -0.80465577,  0.36846605, -0.11969501]])]
[[], array([[-0.25091976,  0.90142861,  0.46398788,  0.19731697, -0.68796272],
       [-0.68801096, -0.88383278,  0.73235229,  0.20223002,  0.41614516],
       [-0.95883101,  0.9398197 ,  0.66488528, -0.57532178, -0.63635007],
       [-0.63319098, -0.39151551,  0.04951286, -0.13610996, -0.41754172]]), array([[ 0.22370579, -0.72101228, -0.4157107 , -0.26727631, -0.08786003],
       [ 0.57035192, -0.60065244,  0.02846888,  0.18482914, -0.90709917],
       [ 0.2150897 , -0.65895175, -0.86989681,  0.89777107,  0.93126407],
       [ 0.6167947 , -0.39077246, -0.80465577,  0.36846605, -0.11969501]])]
[[], array([[-0.25091976,  0.90142861,  0.46398788,  0.19731697, -0.68796272],
       [-0.68801096, -0.88383278,  0.73235229,  0.20223002,  0.41614516],
       [-0.95883101,  0.9398197 ,  0.66488528, -0.57532178, -0.63635007],
       [-0.63319098, -0.39151551,  0.04951286, -0.13610996, -0.41754172]]), array([[ 0.22370579,